# Reservoirs and dams in Spain: attributes
***

**Author:** Chus Casado Rodríguez<br>
**Date:** 28-11-2023<br>

**Introduction:**<br>
In this notebook I extract from the SNCZI (_Sistema Nacional de Cartografía de Zonas Inundables_) attributes for the 394 reservoirs in the data set  _Anuario de Aforos_ (from now on AA). HYDRO requires values of minimum and maximum reservoir outflow and capacity to apply quality checks on the time series in AA.

First I need to find a connection between the reservoirs in AA (point shapefile) and the reservoirs in SNCZI (polygon shapefile). After a try in Python, I saw there are many errors due to the wrong location of the points in comparison with the polygons. Therefore, I decided to do it manually on QGIS since the 394 reservoirs will not take too long.

After that, I have downloaded the XML reports from the reservoirs in SNCZI that match a reservoir in AA and read the attributes from those reports. The resulting table is exported as an CSV file (_attributes_reservoirs.csv_).

Fortunately, the attributes of the SNCZI reservoirs specify the main dam in that reservoir (field _Código de infraestructura_). This attribute allows me to download the XML reports from the dams in SNCZI associated with a reservoir in AA. I read the XML files of attributes and I produce a second table that I export as a CSV file (_attributes_dams.csv_)

**To do:**<br>
* [x] Find connection between the reservoirs in CEDEX and those in SNCZI

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from typing import Union, List, Tuple, Dict
from tqdm.notebook import tqdm
import requests

from utils.SNCZI import reservoir_attributes, dam_attributes

There are three shape files I have extracted from both _AA_ and _SNCZI_. From _AA_ I created a shapefile based on the table _EMBALSE.CSV_. From SNCZI I have downloaded two shapefiles of reservoirs and dams.

In [2]:
path_datasets = Path('Z:/nahaUsers/casadje/datasets/')

## CEDEX

In [3]:
path_cedex = path_datasets / 'CEDEX/processed/reservoirs/attributes/GIS/'

reservoirs_AA = gpd.read_file(path_cedex / 'reservoirs.shp')
reservoirs_AA.ref_ceh = reservoirs_AA.ref_ceh.astype(int)
reservoirs_AA.set_index('ref_ceh', drop=True, inplace=True)
reservoirs_AA.sort_index(axis=0, inplace=True)
reservoirs_AA.to_crs(epsg=25830, inplace=True)

print('AA contains {0} reservoirs'.format(reservoirs_AA.shape[0]))
print('{0} reservoirs are not connected to the SNCZI database'.format((reservoirs_AA.SNCZI == 0).sum()))
# reservoirs_AA.head()

AA contains 394 reservoirs
7 reservoirs are not connected to the SNCZI database


## SNCZI

### Reservoirs

In [6]:
reservoirs_SNCZI = gpd.read_file(path_cedex / 'egis_embalse_geoetrs89.shp')
reservoirs_SNCZI['ID_EMBALSE'] = reservoirs_SNCZI['ID_EMBALSE'].astype(int)
reservoirs_SNCZI.set_index('ID_EMBALSE', drop=True, inplace=True)
reservoirs_SNCZI.sort_index(axis=0, inplace=True)
reservoirs_SNCZI.to_crs(epsg=25830, inplace=True)

print('SNCZI contains {0} reservoirs and {1} attributes'.format(*reservoirs_SNCZI.shape))
# reservoirs_SNCZI.head()

SNCZI contains 3170 reservoirs and 18 attributes


#### Download reports

In [16]:
path_reports = path_datasets / 'SNCZI' / 'reports' / 'reservoirs'
path_reports.mkdir(parents=True, exist_ok=True)

In [8]:
for ID in tqdm(reservoirs_AA.SNCZI):
    
    if ID == 0:
        continue
        
    # output XML file
    filename = f'{path_reports}/{ID:04}.xml'
    if os.path.isfile(filename):
        continue
    
    # extract data from URL
    url = f'https://sig.mapama.gob.es/WebServices/clientews/snczi/Default.aspx?nombre=EGISPE_EMBALSE&claves=ID_EMBALSE&valores={ID}&op=ExpMultiple'
    with requests.get(url) as response:
        lines = [line.decode('utf-8') for line in response.iter_lines()]
    
    # export XML file
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(line + '\n' for line in lines)

  0%|          | 0/394 [00:00<?, ?it/s]

#### Read reports

In [9]:
attributes1 = pd.DataFrame(dtype='object')
for ref_ceh in tqdm(reservoirs_AA.index):
    ID = reservoirs_AA.loc[ref_ceh, 'SNCZI']
    if ID == 0:
        continue
    file = f'{path_reports}/{ID:04}.xml'
    try:
        attributes1 = pd.concat((attributes1, reservoir_attributes(file, name=ref_ceh)), axis=1)
    except:
        print(file)
        continue

attributes1 = attributes1.transpose()
attributes1.index.name = 'ref_ceh'

attributes1.shape

  0%|          | 0/394 [00:00<?, ?it/s]

(387, 19)

In [17]:
# export the reservoirs' attribute table
attributes1.to_csv(path_reports / 'attributes_reservoirs.csv')

### Dams

In [11]:
dams_SNCZI = gpd.read_file(path_cedex / 'egis_presa_geoetrs89.shp')
cols_int = ['ID_INFRAES', 'CODIGO']
dams_SNCZI[cols_int] = dams_SNCZI[cols_int].astype(int)
dams_SNCZI.set_index('ID_INFRAES', drop=True, inplace=True)
dams_SNCZI.sort_index(inplace=True)
dams_SNCZI.to_crs(epsg=25830, inplace=True)

print('SNCZI contains {0} dams and {1} attributes'.format(*dams_SNCZI.shape))
# dams_SNCZI.head()

SNCZI contains 3208 dams and 23 attributes


#### Download reports

In [18]:
path_reports = path_datasets / 'SNCZI' / 'reports' / 'dams'
path_reports.mkdir(parents=True, exist_ok=True)

In [13]:
for ref_ceh in tqdm(reservoirs_AA.index):
    
    # find the ID of the dam
    try:
        code_infr = int(attributes1.loc[ref_ceh, 'Código de infraestructura'])
    except:
        print(ref_ceh)
        continue
    ID = dams_SNCZI.loc[dams_SNCZI.CODIGO == code_infr].index[0]

    # output XML file
    filename = f'{path_reports}/{ID:04}.xml'
    if os.path.isfile(filename):
        continue
    
    # extract data from URL
    url = f'https://sig.mapama.gob.es/WebServices/clientews/snczi/Default.aspx?nombre=EGISPE_PRESA&claves=ID_INFRAESTRUCTURA&valores={ID}&op=Exportar'
    
    with requests.get(url) as response:
        lines = [line.decode('utf-8') for line in response.iter_lines()]
    
    # export XML file
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(line + '\n' for line in lines)

  0%|          | 0/394 [00:00<?, ?it/s]

1274
1346
1357
1360
2028
5059
9834
9853
9854


#### Read reports

In [14]:
attributes2 = pd.DataFrame(dtype='object')
for ref_ceh in tqdm(reservoirs_AA.index):
    
    # find the ID of the dam
    try:
        code_infr = int(attributes1.loc[ref_ceh, 'Código de infraestructura'])
    except:
        print(ref_ceh)
        continue
    ID = dams_SNCZI.loc[dams_SNCZI.CODIGO == code_infr].index[0]
    file = f'{path_reports}/{ID:04}.xml'
    attributes2 = pd.concat((attributes2,dam_attributes(file, name=ref_ceh)), axis=1)

attributes2 = attributes2.transpose()
attributes2.index.name = 'ref_ceh'

attributes2.shape

  0%|          | 0/394 [00:00<?, ?it/s]

1274
1346
1357
1360
2028
5059
9834
9853
9854


(385, 36)

In [19]:
# export the dams' attribute table
attributes2.to_csv(path_reports / 'attributes_dams.csv')

## Mapping
### Map reservoir and dams in SNCZI

The ID of the reservoirs do not match between AA and SNCZI, and not even the reservoirs and the dams in SNCZI. Many reservoirs in SNCZI have more than one dam associated.

```Python
buffer_distance = 100
attrs = ['AP_M_ANUAL', 'NMN_CAPAC', 'NMN_SUP', 'NOMBRE', 'SUP_CUENCA']#, 'PROVINCIA', 'TITULAR']
map_SNCZI = {}
for ID_EMBALSE, reservoir in tqdm(reservoirs_SNCZI.iterrows()):
    # Create a buffer around the polygon
    buffer = reservoir['geometry'].buffer(buffer_distance)
    
    # Find points in dams_SNCZI that are within the buffer
    dams = dams_SNCZI[dams_SNCZI['geometry'].within(buffer)]
    mask = (reservoir[attrs] == dams[attrs]).any(axis=1)
    if mask.sum() > 0:
        # Store the indices of the points in the result dictionary
        map_SNCZI[ID_EMBALSE] = dams[mask].index.tolist()
    else:
        print(f'No dams were associated to reservoir {ID_EMBALSE}')

print('Reservoirs in the mapping:\t{0}'.format(len(map_SNCZI)))
print('Reservoirs in SNCZI:\t\t{0}'.format(reservoirs_SNCZI.shape[0]))
print()
ids = [id for id_list in map_SNCZI.values() for id in id_list]
print('Total of dams in the mapping:\t{0}'.format(len(ids)))
print('Unique dams in the mapping:\t{0}'.format(len(set(ids))))
print('Dams in SNCZI:\t\t\t{0}'.format(dams_SNCZI.shape[0]))
```

There are several dams that are attributed to more than 1 reservoir, and some dams that are not attributed to any reservoirs.

```Python
from collections import Counter

counts = pd.Series(Counter(ids))
counts = counts.sort_values(ascending=False)

counts.head()
```

### Map AA with SNCZI

```Python
map_ids = {}
reservoirs_AA['SNCZI_ID'] = np.nan
for ref_ceh in tqdm(reservoirs_AA.index):
    ID = None
    name, catchment = reservoirs_AA.loc[ref_ceh, ['nombre', 'cuenca']]
    mask = dams.NOMBRE.str.contains(name)# & dams.DEMARC.str.contains(catchment)
    aux = dams.loc[mask].copy()
    if aux.shape[0] == 0:
        gdf1 = reservoirs_AA.loc[[ref_ceh]].copy()
        gdf1['geometry'] = gdf1['geometry'].buffer(distance)
        aux = gpd.sjoin(gdf1, dams, predicate='intersects', how='inner')
        aux['ref_ceh'] = aux.index
        aux.set_index('index_right', inplace=True)
        # if joined.shape[0] > 0:
        #     ID = joined.NMN_CAPAC.idxmax()
    if aux.shape[0] == 1:
        ID = aux.index[0]
    elif aux.shape[0] > 1:
        try:
            aux.CATEGORIA.replace({'A': 1, 'B': 2, 'C': 3}, inplace=True)
            ID = aux.CATEGORIA.idxmin()
        except:
            ID = aux.NMN_CAPAC.idxmin()
    if ID is not None:
        reservoirs_AA.loc[ref_ceh, 'SNCZI_ID'] = ID
        map_ids[ID] = ref_ceh
```